In [1]:
%pip install gender_guesser

     |████████████████████████████████| 379 kB 5.2 MB/s 


In [21]:
import os
import json
import time
import requests
import datetime
import dateutil
import glob
import pandas as pd
import gender_guesser.detector as gender
from dateutil.relativedelta import relativedelta
from datetime import datetime

In [3]:
key = '1bNDsdkLtXfBKAwBjKImjegea8H8JqKa'
end_year = 2020
num_years = 10
end = datetime.date(end_year, 12, 31)
start = end - relativedelta(years=num_years)
dates = [x.split(' ') for x in pd.date_range(start, end, freq='MS').strftime("%Y %-m").tolist()]
print(dates)

[['2011', '1'], ['2011', '2'], ['2011', '3'], ['2011', '4'], ['2011', '5'], ['2011', '6'], ['2011', '7'], ['2011', '8'], ['2011', '9'], ['2011', '10'], ['2011', '11'], ['2011', '12'], ['2012', '1'], ['2012', '2'], ['2012', '3'], ['2012', '4'], ['2012', '5'], ['2012', '6'], ['2012', '7'], ['2012', '8'], ['2012', '9'], ['2012', '10'], ['2012', '11'], ['2012', '12'], ['2013', '1'], ['2013', '2'], ['2013', '3'], ['2013', '4'], ['2013', '5'], ['2013', '6'], ['2013', '7'], ['2013', '8'], ['2013', '9'], ['2013', '10'], ['2013', '11'], ['2013', '12'], ['2014', '1'], ['2014', '2'], ['2014', '3'], ['2014', '4'], ['2014', '5'], ['2014', '6'], ['2014', '7'], ['2014', '8'], ['2014', '9'], ['2014', '10'], ['2014', '11'], ['2014', '12'], ['2015', '1'], ['2015', '2'], ['2015', '3'], ['2015', '4'], ['2015', '5'], ['2015', '6'], ['2015', '7'], ['2015', '8'], ['2015', '9'], ['2015', '10'], ['2015', '11'], ['2015', '12'], ['2016', '1'], ['2016', '2'], ['2016', '3'], ['2016', '4'], ['2016', '5'], ['2016', 

In [4]:
#@title Gendered Words
gendered_words = {
  "mr": "ms",
  "brother-in-laws": "hers",
  "father-in-laws": "mrs",
  "he": "she",
  "he's": "she's",
  "fiancé": "fiancée",
  "his": "her",
  "son-in-law": "daughter-in-law",
  "sons-in-law": "daughters-in-law",
  "brother-in-law": "sister-in-law",
  "brothers-in-law": "sisters-in-law",
  "father-in-law": "mother-in-law",
  "fathers-in-law": "mothers-in-law",
  "him": "her",
  "himself": "herself",
  "lawman": "lawwoman",
  "daddy" : "mommy",
  "man": "woman",
  "men": "women",
  "nephew": "niece",
  "nephews": "nieces",
  "father": "mother",
  "fathers": "mothers",
  "brother": "sister",
  "brothers": "sisters",
  "brother": "goddess",
  "brothers": "godesses",
  "uncle": "aunt",
  "uncles": "aunts",
  "male": "female",
  "males": "females",
  "mister": "madam",
  "duke": "duchess",
  "dukes": "duchesses",
  "conductor": "conductress",
  "conductors": "conductresses",
  "sir": "madame",
  "actor": "actress",
  "actors": "actresses",
  "boy": "girl",
  "boys": "girls",
  "congressman": "congresswoman",
  "congressmen": "congresswomen",
  "wizard": "wizardess",
  "wizards": "wizardesses",
  "witcher": "witch",
  "witchers": "witches",
  "lord": "lady",
  "lords": "ladies",
  "guy": "gal",
  "guys": "gals",
  "groom": "bride",
  "grooms": "brides",
  "king": "queen",
  "kings": "queens",
  "hero": "heroine",
  "heroes": "heroines",
  "policeman": "policewoman",
  "policemen": "policewomen",
  "husband": "wife",
  "husbands": "wives",
  "son": "daughter",
  "sons": "daughters",
  "grandson": "granddaughter",
  "grandsons": "granddaughters",
  "dad": "mom",
  "dads": "moms",
  "he's": "she's",
  "heir": "heiress",
  "heirs": "heiresses",
  "baron": "baroness",
  "barons": "baronesses",
  "prince": "princess",
  "princes": "princesses",
  "emperor": "empress",
  "emperors": "empresses",
  "tsar": "tsarina",
  "tsars": "tsarinas",
  "czar": "czarina",
  "czars": "czarinas",
  "father": "mother",
  "fathers": "mothers",
  "grandfather": "grandmother",
  "grandfathers": "grandmothers",
  "greatgrandfather": "greatgrandmother",
  "greatgrandfathers": "greatgrandmothers",
  "waiter": "waitress",
  "waiters": "waitresses",
  "master": "mistress",
  "masters": "mistresses",
  "chairman": "chairwoman",
  "chairmen": "chairwomen",
  "headmaster": "headmistress",
  "headmasters": "headmistresses",
  "landlord": "landlady",
  "landlords": "landladies",
  "widower": "widow",
  "widowers": "widows",
  "gentleman": "lady",
  "gentlemen": "ladies",
  "steward": "stewardess",
  "steward": "stewardesses",
  "masculine": "feminine",
  "fireman": "firewoman",
  "firemen": "firewomen",
  "waterman": "waterwoman",
  "watermen": "waterwomen",
  "nozzleman": "nozzlewoman",
  "nozzlemen": "nozzlewomen",
  "adman": "adwoman",
  "admen": "adwomen",
  "agribusinessman": "agribusinesswoman",
  "agribusinessmen": "agribusinesswomen",
  "aidman": "aidwoman",
  "airmen": "aidwomen",
  "alderman": "alderwoman",
  "aldermen": "alderwomen",
  "almsman": "almswoman",
  "almsmen": "almswomen",
  "anchorman": "anchorwoman",
  "anchormen": "anchorwomen",
  "antiman": "antiwoman",
  "antimen": "antiwomen",
  "artilleryman": "artillerywoman",
  "artillerymen": "artillerywomen",
  "ashmen": "ashwomen",
  "assemblyman": "assemblywoman",
  "assemblymen": "assemblywomen",
  "ataman": "atawoman",
  "atamen": "atawomen",
  "attackman": "attackwoman",
  "attackmen": "attackwomen",
  "automan": "autowoman",
  "automae": "autowomen",
  "axeman": "axewoman",
  "axemen": "axewomen",
  "axman": "axwoman",
  "axmen": "axwomen",
  "backcourtman": "backcourtwoman",
  "backcourtmen": "backcourtwomen",
  "backwoodsman": "backwoodswoman",
  "backwoodsmen": "backwoodswomen",
  "badman": "badwoman",
  "badmen": "badwomen",
  "bagman": "bagwoman",
  "bagmen": "bagwomen",
  "bandsman": "bandswoman",
  "bandsmen": "bandswomen",
  "bargeman": "bargewoman",
  "bargemen": "bargewomen",
  "barman": "barwoman",
  "barmen": "barwomen",
  "baseman": "basewoman",
  "basemen": "basewomen",
  "batman": "batwoman",
  "batmen": "batwomen",
  "batsman": "batswoman",
  "batsmen": "batswomen",
  "bayman": "baywoman",
  "baymen": "baywomen",
  "beadsman": "beadswoman",
  "beadsmen": "beadswomen",
  "bedesman": "bedeswoman",
  "bedesmen": "bedeswomen",
  "bellman": "bellwoman",
  "bellmen": "bellwomen",
  "birdman": "birdwoman",
  "birdmen": "birdwomen",
  "bluesman": "blueswoman",
  "bluesmen": "blueswomen",
  "boardman": "boardwoman",
  "boardmen": "boardwomen",
  "boatman": "boatwoman",
  "boatmen": "boatwomen",
  "boatsman": "boatswoman",
  "boatsmen": "boatswomen",
  "bogyman": "bogywoman",
  "bogymen": "bogywomen",
  "bondman": "bondwoman",
  "bondmen": "bondwomen",
  "bondsman": "bondswomen",
  "bondsmen": "bondswoman",
  "boogerman": "boogerwoman",
  "boogermen": "boogerwomen",
  "boogeyman": "boogeywoman",
  "boogeymen": "boogeywomen",
  "boogyman": "boogywoman",
  "boogymen": "boogywomen",
  "bookman": "bookwoman",
  "bookmen": "bookwomen",
  "bowmen": "bowwomen",
  "brakeman": "brakewoman",
  "brakemen": "brakewomen",
  "bushman": "bushwoman",
  "bushmen": "bushwomen",
  "businessman": "businesswoman",
  "businessmen": "businesswomen",
  "busman": "buswoman",
  "busmen": "buswomen",
  "cabman": "cabwoman",
  "cabmen": "cabwomen",
  "cameraman": "camerawoman",
  "cameramen": "camerawomen",
  "carman": "carwoman",
  "carmen": "carwomen",
  "cattleman": "cattlewoman",
  "cattlemen": "cattlewomen",
  "cavalryman": "cavalrywoman",
  "cavalrymen": "cavalrywomen",
  "caveman": "cavewoman",
  "cavemen": "cavewoman",
  "cayman": "caywoman",
  "caymen": "caywomen",
  "chainman": "chainwoman",
  "chainmen": "chainwomen",
  "chairmen": "chairwomen",
  "chapmen": "chapwomen",
  "chessman": "chesswoman",
  "chessmen": "chesswomen",
  "choreman": "chorewoman",
  "choremen": "chorewomen",
  "churchman": "churchwoman",
  "churchmen": "churchwomen",
  "clansman": "clanswoman",
  "clansmen": "clanswomen",
  "clergyman": "clergywoman",
  "clergymen": "clergywomen",
  "clubman": "clubwoman",
  "clubmen": "clubwomen",
  "coachman": "coachwoman",
  "coachmen": "coachwomen",
  "coastguardman": "coastguardwoman",
  "coastguardmen": "coastguardwomen",
  "coastguardsman": "coastguardswoman",
  "coastguardsmen": "coastguardswomen",
  "cochairman": "cochairwoman",
  "cochairmen": "cochairwomen",
  "colorman": "colorwoman",
  "colormen": "colorwomen",
  "committeeman": "committeewoman",
  "committeemen": "committeewomen",
  "cornerman": "cornerwoman",
  "cornermen": "cornerwomen",
  "corpsman": "corpswoman",
  "corpsmen": "corpswomen",
  "councilman": "councilwoman",
  "councilmen": "councilwomen",
  "counterman": "counterwoman",
  "countermen": "counterwomen",
  "countryman": "countrywoman",
  "countrymen": "countrywomen",
  "cowman": "cowwoman",
  "cowmen": "cowwomen",
  "cracksman": "crackswoman",
  "cracksmen": "crackswomen",
  "craftsman": "craftswoman",
  "craftsmen": "craftswomen",
  "cragsman": "cragswoman",
  "cragsmen": "cragswomen",
  "crewman": "crewwoman",
  "crewmen": "crewwomen",
  "crossbowman": "crossbowwoman",
  "crossbowmen": "crossbowwomen",
  "dairyman": "dairywoman",
  "dairymen": "dairywomen",
  "dalesmen": "daleswomen",
  "damen": "dawomen",
  "daysman": "dayswoman",
  "daysmen": "dayswomen",
  "deathsman": "deathswoman",
  "deathsmen": "deathswomen",
  "decumen": "decuwomen",
  "everyman": "everywoman",
  "everymen": "everywomen",
  "exciseman": "excisewoman",
  "excisemen": "excisewomen",
  "expressman": "expresswoman",
  "expressmen": "expresswomen",
  "firemen": "firewomen",
  "fisherman": "fisherwoman",
  "fishermen": "fisherwomen",
  "footman": "footwoman",
  "footmen": "footwomen",
  "frontman": "frontwoman",
  "frontmen": "frontwomen",
  "funnyman": "funnywoman",
  "funnymen": "funnywomen",
  "guardsman": "guardswoman",
  "guardsmen": "guardswomen",
  "highwayman": "highwaywoman",
  "highwaymen": "highwaywomen",
  "horseman": "horsewoman",
  "horsemen": "horsewomen",
  "hotelman": "hotelwoman",
  "hotelmen": "hotelwomen",
  "houseman": "housewoman",
  "housemen": "housewomen",
  "iceman": "icewoman",
  "icemen": "icewomen",
  "jazzman": "jazzwoman",
  "jazzmen": "jazzwomen",
  "journeyman": "journeywoman",
  "journeymen": "journeywomen",
  "kinsman": "kinswoman",
  "kinsmen": "kinswomen",
  "landman": "landwoman",
  "landmen": "landwomen",
  "lobsterman": "lobsterwoman",
  "lobstermen": "lobsterwomen",
  "madman": "madwoman",
  "madmen": "madwomen",
  "mailman": "mailwoman",
  "mailmen": "mailwomen",
  "marksman": "markswoman",
  "marksmen": "markswomen",
  "meatman": "meatwoman",
  "meatmen": "meatwomen",
  "merchantman": "merchantwoman",
  "merchantmen": "merchantwomen",
  "merman": "mermaid",
  "mermen": "mermaids",
  "messman": "messwoman",
  "messmen": "messwomen",
  "middleman": "middlewoman",
  "middlemen": "middlewomen",
  "midshipman": "midshipwoman",
  "midshipmen": "midshipwomen",
  "militiaman": "militiawoman",
  "militiamen": "militiawomen",
  "milkman": "milkwoman",
  "milkmen": "milkwomen",
  "minuteman": "minutewoman",
  "minutemen": "minutewomen",
  "missileman": "missilewoman",
  "missilemen": "missilewomen",
  "moneyman": "moneywoman",
  "moneymen": "moneywomen",
  "motorman": "motorwoman",
  "motormen": "motorwomen",
  "newsman": "newswoman",
  "newsmen": "newswomen",
  "newspaperman": "newspaperwoman",
  "newspapermen": "newspaperwomen",
  "nobleman": "noblewoman",
  "noblemen": "noblewomen",
  "nonman": "nonwoman",
  "nonmen": "nonwomen",
  "ottomen": "ottowomen",
  "outdoorsman": "outdoorswoman",
  "outdoorsmen": "outdoorswomen",
  "overman": "overwoman",
  "pivotman": "pivotwoman",
  "placeman": "placewoman",
  "plainclothesman": "plainclotheswoman",
  "plainsman": "plainswoman",
  "plantsman": "plantswoman",
  "plowman": "plowwoman",
  "pointman": "pointwoman",
  "postman": "postwoman",
  "potman": "potwoman",
  "poultryman": "poultrywoman",
  "prefreshman": "prefreshwoman",
  "quarryman": "quarrywoman",
  "radioman": "radiowoman",
  "raftsman": "raftswoman",
  "ragman": "ragwoman",
  "ranchman": "ranchwoman",
  "reinsman": "reinswoman",
  "repairman": "repairwoman",
  "rifleman": "riflewoman",
  "rodsman": "rodswoman",
  "roundsman": "roundswoman",
  "routeman": "routewoman",
  "safetyman": "safetywoman",
  "sagaman": "sagawoman",
  "salaryman": "salarywoman",
  "salesman": "saleswoman",
  "sandman": "sandwoman",
  "schoolman": "schoolwoman",
  "seaman": "seawoman",
  "seedsman": "seedswoman",
  "selectman": "selectwoman",
  "shopman": "shopwoman",
  "showman": "showwoman",
  "sideman": "sidewoman",
  "signalman": "signalwoman",
  "skyman": "skywoman",
  "snowman": "snowwoman",
  "spaceman": "spacewoman",
  "spokesman": "spokeswoman",
  "sportfisherman": "sportfisherwoman",
  "sportsman": "sportswoman",
  "statesman": "stateswoman",
  "stickman": "stickwoman",
  "stillman": "stillwoman",
  "stockman": "stockwoman",
  "outmen": "outwomen",
  "overmen": "overwomen",
  "pivotmen": "pivotwomen",
  "placemen": "placewomen",
  "plainclothesmen": "plainclotheswomen",
  "plainsmen": "plainswomen",
  "plantsmen": "plantswomen",
  "plowmen": "plowwomen",
  "pointmen": "pointwomen",
  "policemen": "policewomen",
  "postmen": "postwomen",
  "potmen": "potwomen",
  "poultryman": "poultrywoman",
  "poultrymen": "poultrywomen",
  "prefreshman": " prefreshwoman",
  "prefreshmen": " prefreshwomen",
  "pullman": "pullwoman",
  "pullmen": "pullwomen",
  "quarryman": "quarrywoman",
  "quarrymen": "quarrywomen",
  "radioman": "radiowoman",
  "radiomen": "radiowomen",
  "raftsman": "raftswoman",
  "raftsmen": "raftswomen",
  "ragman": "ragwoman",
  "ragmen": "ragwomen",
  "ranchman": "ranchwoman",
  "ranchmen": "ranchwomen",
  "reedman": "reedwoman",
  "reedmen": "reedwomen",
  "reinsman": "reinswoman",
  "reinsmen": "reinswomen",
  "reman": "rewoman",
  "remen": "rewomen",
  "repairman": "repairwoman",
  "repairmen": "repairwomen",
  "rifleman": "riflewoman",
  "riflemen": "riflewomen",
  "rodman": "rodwoman",
  "rodmen": "rodwomen",
  "rodsman": "rodswoman",
  "rodsmen": "rodswomen",
  "romen": "rowomen",
  "roundsman": "roundswoman",
  "roundsmen": "roundswomen",
  "routeman": "routewoman",
  "routemen": "routewomen",
  "safetyman": "safetywoman",
  "safetymen": "safetywomen",
  "sagaman": "sagawoman",
  "sagamen": "sagawomen",
  "salaryman": "salarywoman",
  "salarymen": "salarywomen",
  "salesman": "saleswoman",
  "salesmen": "saleswomen",
  "sandman": "sandwoman",
  "sandmen": "sandwomen",
  "schoolmen": "schoolwomen",
  "schoolman": "schoolwoman",
  "seaman": "seawoman",
  "seamen": "seawomen",
  "seedsman": "seedswoman",
  "seedsmen": "seedswomen",
  "selectman": "selectwoman",
  "selectmen": "selectwomen",
  "shipman": "shipwoman",
  "shipmen": "shipwomen",
  "shopman": "shopwoman",
  "shopmen": "shopwomen",
  "showman": "showwoman",
  "showmen": "showwomen",
  "sideman": "sidewoman",
  "sidemen": "sidewomen",
  "signalman": "signalwoman",
  "signalmen": "signalwomen",
  "skyman": "skywoman",
  "skymen": "skywomen",
  "snowman": "snowwoman",
  "snowmen": "snowwomen",
  "sockman": "sockwoman",
  "sockmen": "sockwomen",
  "soundman": "soundwoman",
  "soundmen": "soundwomen",
  "spaceman": "spacewoman",
  "spacemen": "spacewomen",
  "spokesman": "spokeswoman",
  "spokesmen": "spokeswomen",
  "sportfisherman": "sportfisherwoman",
  "sportfishermen": "sportfisherwomen",
  "sportsman": "sportswoman",
  "sportsmen": "sportswomen",
  "statesmen": "statesman",
  "stickman": "stickwoman",
  "stickmen": "stickwomen",
  "stockman": "stockwoman",
  "stockmen": "stockwomen",
  "strongman": "strongwoman",
  "strongmen": "strongwomen",
  "superman": "superwoman",
  "supermen": "superwomen",
  "supersalesman": "supersaleswoman",
  "supersalesmen": "supersaleswomen",
  "vanman": "vanwoman",
  "vanmen": "vanwomen",
  "venireman": "venirewoman",
  "veniremen": "venirewomen",
  "workingman": "workingwoman",
  "workingmen": "workingwomen",
  "workman": "workwoman",
  "workmen": "workwomen",
  "yachtman": "yachtwoman",
  "yachtmen": "yachtwomen",
  "yachtsman": "yachtswoman",
  "yachtsmen": "yachtswomen",
  "yardman": "yardwoman",
  "yardmen": "yardwomen"
}

In [5]:
male_words = list(gendered_words.keys())
female_words = list(gendered_words.values())
no_gender = ['the', 'The', 'and', 'And']
d = gender.Detector(case_sensitive=False)

def gender_checker(word):
  if word not in no_gender:
    if d.get_gender(word) in ['male', 'mostly_male'] or word in male_words:
      return "m"
    if d.get_gender(word) in ['female', 'mostly_female'] or word in female_words:
      return "f"

In [6]:
def send_request(date):
    '''Sends a request to the NYT Archive API for given date.'''
    base_url = 'https://api.nytimes.com/svc/archive/v1/'
    url = base_url + '/' + date[0] + '/' + date[1] + '.json?api-key=' + key
    response = requests.get(url).json()
    time.sleep(6)
    return response


def is_valid(article, date):
    '''An article is only worth checking if it is in range, and has a headline.'''
    is_in_range = date > start and date < end
    has_headline = type(article['headline']) == dict and 'main' in article['headline'].keys()
    has_snippet = len(article['snippet']) > 0
    return is_in_range and has_headline and has_snippet


def parse_response(response):
    '''Parses and returns response as pandas data frame.'''
    data = {  
        'date': [],
        'm_to_f_percentage': []
        }
    
    articles = response['response']['docs'] 
    for article in articles: # For each article, make sure it falls within our date range
        date = dateutil.parser.parse(article['pub_date']).date()
        if is_valid(article, date):
          if article['document_type'] == 'article':
            male_count = 0
            fem_count = 0
            data['date'].append(date)
            #data['snippet'].append(article['snippet'])
            for word in article['snippet'].split():
              if gender_checker(word) == 'm':
                male_count += 1
              if gender_checker(word) == 'f':
                fem_count += 1
            total_count = male_count + fem_count
            if total_count == 0:
              percent = 0.50
            else:
              percent = round(male_count / (male_count + fem_count),2)
            data['m_to_f_percentage'].append(percent)
            #data['male_count'].append(male_count)
            #data['fem_count'].append(fem_count)
            #data['headline'].append(article['headline']['main']) 
            #keywords = [keyword['value'] for keyword in article['keywords'] if keyword['name'] == 'subject']
            #data['keywords'].append(keywords)

    return pd.DataFrame(data) 


def get_data(dates):
    '''Sends and parses request/response to/from NYT Archive API for given dates.'''
    total = 0
    print('Date range: ' + str(dates[0]) + ' to ' + str(dates[-1]))
    if not os.path.exists('headlines'):
        os.mkdir('headlines')
    for date in dates:
        response = send_request(date)
        df = parse_response(response)
        total += len(df)
        df.to_csv('headlines/' + date[0] + '-' + date[1] + '.csv', index=False)
        print('Saving headlines/' + date[0] + '-' + date[1] + '.csv...')
    print('Number of articles collected: ' + str(total))

In [7]:
get_data(dates)

Date range: ['2011', '1'] to ['2020', '12']
Saving headlines/2011-1.csv...
Saving headlines/2011-2.csv...
Saving headlines/2011-3.csv...
Saving headlines/2011-4.csv...
Saving headlines/2011-5.csv...
Saving headlines/2011-6.csv...
Saving headlines/2011-7.csv...
Saving headlines/2011-8.csv...
Saving headlines/2011-9.csv...
Saving headlines/2011-10.csv...
Saving headlines/2011-11.csv...
Saving headlines/2011-12.csv...
Saving headlines/2012-1.csv...
Saving headlines/2012-2.csv...
Saving headlines/2012-3.csv...
Saving headlines/2012-4.csv...
Saving headlines/2012-5.csv...
Saving headlines/2012-6.csv...
Saving headlines/2012-7.csv...
Saving headlines/2012-8.csv...
Saving headlines/2012-9.csv...
Saving headlines/2012-10.csv...
Saving headlines/2012-11.csv...
Saving headlines/2012-12.csv...
Saving headlines/2013-1.csv...
Saving headlines/2013-2.csv...
Saving headlines/2013-3.csv...
Saving headlines/2013-4.csv...
Saving headlines/2013-5.csv...
Saving headlines/2013-6.csv...
Saving headlines/201

In [8]:
!zip -r /content/nyt_data.zip /content/headlines

  adding: content/headlines/ (stored 0%)
  adding: content/headlines/2012-2.csv (deflated 95%)
  adding: content/headlines/2016-3.csv (deflated 95%)
  adding: content/headlines/2013-12.csv (deflated 95%)
  adding: content/headlines/2019-11.csv (deflated 95%)
  adding: content/headlines/2011-6.csv (deflated 95%)
  adding: content/headlines/2019-9.csv (deflated 95%)
  adding: content/headlines/2019-2.csv (deflated 95%)
  adding: content/headlines/2012-5.csv (deflated 95%)
  adding: content/headlines/2017-12.csv (deflated 95%)
  adding: content/headlines/2014-10.csv (deflated 95%)
  adding: content/headlines/2018-9.csv (deflated 95%)
  adding: content/headlines/2018-1.csv (deflated 95%)
  adding: content/headlines/2011-7.csv (deflated 95%)
  adding: content/headlines/2017-10.csv (deflated 95%)
  adding: content/headlines/2020-10.csv (deflated 95%)
  adding: content/headlines/2016-8.csv (deflated 95%)
  adding: content/headlines/2017-7.csv (deflated 95%)
  adding: content/headlines/2018-6.

In [ ]:
#!rm -rf headlines

In [9]:
!unzip nyt_data.zip

Archive:  nyt_data.zip
   creating: content/headlines/
  inflating: content/headlines/2012-2.csv  
  inflating: content/headlines/2016-3.csv  
  inflating: content/headlines/2013-12.csv  
  inflating: content/headlines/2019-11.csv  
  inflating: content/headlines/2011-6.csv  
  inflating: content/headlines/2019-9.csv  
  inflating: content/headlines/2019-2.csv  
  inflating: content/headlines/2012-5.csv  
  inflating: content/headlines/2017-12.csv  
  inflating: content/headlines/2014-10.csv  
  inflating: content/headlines/2018-9.csv  
  inflating: content/headlines/2018-1.csv  
  inflating: content/headlines/2011-7.csv  
  inflating: content/headlines/2017-10.csv  
  inflating: content/headlines/2020-10.csv  
  inflating: content/headlines/2016-8.csv  
  inflating: content/headlines/2017-7.csv  
  inflating: content/headlines/2018-6.csv  
  inflating: content/headlines/2011-9.csv  
  inflating: content/headlines/2019-4.csv  
  inflating: content/headlines/2019-7.csv  
  inflating: co

In [62]:
def parse_data():
  data = {  
        'date': [],
        'average_ratio': []
        }
  monthly_data = glob.glob('/content/content/headlines'+ '/*.csv')
  monthly_data = sorted(monthly_data, key = lambda x: (int(x[27:-4][0:4]), int(x[27:-4][5:])))
  for file in monthly_data:
    table = pd.read_csv(file)
    average = table['m_to_f_percentage'].mean()
    data['date'].append(file[27:-4])
    data['average_ratio'].append(average)
  return pd.DataFrame(data)

def get_ratios():
  if not os.path.exists('ratios'):
    os.mkdir('ratios')
  df = parse_data()
  df.to_csv('ratios/' + 'nyt_ratios.csv', index=False)
  print('Saving ratios/nyt_ratios.csv...')

In [63]:
get_ratios()

Saving ratios/nyt_ratios.csv...
